# Install packages

First thing is to install pandas, numpy, scikit-learn, matplotlib, and seaborn. Then below you see where we import the packages in our notebook.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, ElasticNet, Lasso
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, plot_confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler



# Import the data and start exploring it

In [ ]:
data = pd.read_excel('../Project Retain Alabama/Survey Data and Info/UAB_Retain Alabama Survey Data_042221.xlsx')

pd.options.display.max_columns = None

data.head()

Some commands you could start with : 
- data.describe()
- data.shape
- data.isnull().sum()
- data['column name'].value_counts()

# Deleting Date Columns as they seem useless




In [ ]:
drop_colums = ['Start Date','End Date', '0']
data.drop(drop_colums, inplace=True, axis=1)
data.head()


# Transform the Data for Regression

In [ ]:
# Use get dummies for one hot encoding 
dummies = data['col name'].str.get_dummies(sep =',')


In [ ]:
# A useful function for changing strings 
for i in list(data.columns):
    data[i] = data[i].apply(lambda x: str(x)).apply(lambda x: x.replace('|' , ','))

In [ ]:
# renaming columns
data = data.rename(columns = {'old_name': 'new_name','old_name': 'new_name'})

# Regression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

# Model Validation

# 